<a href="https://colab.research.google.com/github/bhuvanmdev/hackorama/blob/master/notebooks/hack_o_rama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install bitsandbytes
# !pip install -qqq torch==2.0.1 --progress-bar off
# !pip uninstall transformers
# !pip install transformers
!pip install transformers
!pip install accelerate
!pip install xformers
!pip install einops
!pip install langchain
# !pip install tensorflow>=2.14
!jupyter notebook restart
!pip install -q -U safetensors
!pip install -q -U peft




In [ ]:
import torch
from transformers import BitsAndBytesConfig,AutoModelForCausalLM, AutoTokenizer,pipeline
from flask import Flask, render_template, request,jsonify ,send_file
import requests
from pyngrok import ngrok,conf
from flask_cors import CORS
from google.colab import userdata

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
                                                        # down one version with smaller chunks on safetensors for low RAM environments
model_id = 'OpenAssistant/codellama-13b-oasst-sft-v10'# "vilsonrodrigues/falcon-7b-instruct-sharded"
# from transformers import AwqConfig
model_4bit_openA = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        # load_in_8bit=True,
        quantization_config=quantization_config,
        trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
pipeline = transformers.pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=1024,
        do_sample=True,
        top_k=3,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)


In [ ]:
open_chatbot_template = """
<|im_start|>system
You are Max, a virtual assistant designed to provide efficient customer support for an ecommerce website called broshop. Users rely on your assistance to find and know products in their cart, and receive clear and direct answers.
the website is a platform for sellers on broshop, a wholesale marketplace that connects buyers and sellers across India. Your website allows sellers to manage their products, orders, payments, and customer service through a simple and intuitive interface. On the left side of the homepage, there is a sidebar with icons and labels for different sections, such as:
Products = contains sections to add product(either single product or bulk product).
Orders = contains history of all the orders made by the user.
Calculator = used to do calculations.
Remittance = contains finished orders.
Help desk = used to raise an issue or checked the status of an already raised issue.
FAQ = used to see FAQ or ask a question.
Profile = user detials including personal and account details.
Logout = logs out the user.
Each option has an icon and a brief instruction to guide the sellers.
contact info = bro@mail.com +1-555-5555
hq = las vegas

If you don't know the answer to a question, please don't share false information.<|im_end|>
<|im_start|>user
how do i change my user name?<|im_end|>
<|im_start|>assistant
"""

new_open = """
<|im_start|>system
Primary prompt instruction: Given a table of product information, write a paragraph describing the product and upload it to the website.

ReAct steps:
Interleave thought, action, and observation steps.
Thought can reason about the current situation and the goal state.
Action can select one of the following commands: enter, upload, select, or write.
Observation can report the result of the action and the current state of the website.

Example:
Thought: Let’s start by entering the product title.
Action: enter “Handmade Wooden Clock” in the title field.
Observation: The title field now shows “Handmade Wooden Clock”.
Thought: Next, let’s upload some images of the product.
Action: upload the first image from the table.
Observation: The first image slot now shows the image of the product.
Thought: Let’s continue uploading the rest of the images.
Action: upload the second image from the table.
Observation: The second image slot now shows the image of the product.
Thought: Now, let’s select the category and subcategory of the product.
Action: select “Home Decor” from the category dropdown menu.
Observation: The category field now shows “Home Decor”.
Action: select “Clocks” from the subcategory dropdown menu.
Observation: The subcategory field now shows “Clocks”.
Thought: Next, let’s enter the prices and the minimum order quantity of the product.
Action: enter “1500” in the maximum retail price field.
Observation: The maximum retail price field now shows “1500”.
Action: enter “1000” in the selling price field.
Observation: The selling price field now shows “1000”.
Action: enter “1” in the minimum order quantity field.
Observation: The minimum order quantity field now shows “1”.
Thought: Next, let’s write a description of the product using the information from the table.
Action: write “This is a handmade wooden clock that will add a rustic charm to your home. It is made of high-quality wood and has a natural finish. The clock has a diameter of 12 inches and runs on a single AA battery. It is easy to hang on the wall and comes with a warranty of 6 months.”
Observation: The description field now shows the written text.
Thought: Finally, let’s enter the material and size of the product.
Action: enter “Wood” in the material field.
Observation: The material field now shows “Wood”.
Action: enter “12 inches” in the size field.
Observation: The size field now shows “12 inches”.
Thought: We have completed the product creation process. Let’s upload the product to the website.
Action: click on the upload button.
Observation: The product is successfully uploaded to the website.<|im_end|>
<|im_start|>user
add a product titled jeans under jean category with mrp 20, selling price 10 , no description, 20 days of dispatch.?<|im_end|>
<|im_start|>assistant
"""

order_placing = '''
<|im_start|>system
you are an AI assistant that helps to extract 8 features namely title,category,mrp,selling cost,material,size,delivery date and description. from the user's input and display it to them pointwise. if any of the feature is not present in the input, you truthfully say that the info is not specified.<|im_end|>
<|im_start|>user
Add a product with title jeans of clothing category of 30 dollar mrp with 20 dollar selling cost. It was made of linen of length 32. It should arrive in 2 days. description not required<|im_end|>
<|im_start|>assistant
title: jeans
category: clothing
mrp: 30 dollar
selling price: 20 dollar
material: linen
size: 32
delivery date: 2 days
description: not specified
order placed successfully.<|im_end|>
<|im_start|>user
Add a product with title sneakers of footwear category priced at 50 mrp and selling for 40. These stylish shoes are made of synthetic material and come in various sizes. Expect delivery in 3 days.<|im_end|>
<|im_start|>assistant
'''

raise_issue = '''
<|im_start|>system
you are an AI assistant that help to extract 2 features namely category and description from the user's input and display it to them pointwise. if any of the feature is not present in the input, you truthfully say that the info is not specified.<|im_end|>
<|im_start|>user
raise an issue on regarding the late arrival of my goods on 8th april 2009 which was ordered in 10th march 2009 and was promissed to be deliverd in 2 days.my order contained 3 pair of socks.<|im_end|>
<|im_start|>assistant
category: delivary
description: I am expressing my concern about the delayed arrival of my goods. I mention that I placed the order on March 10, 2009, with the expectation of receiving it within 2 days. However, the goods did not arrive as promised, and I specifically note the delivery date as April 8, 2009. The order consisted of three pairs of socks. I am seeking assistance or resolution by raising an issue regarding this delay.
task done successfully! please wait till our team assist you.<|im_end|>
<|im_start|>user
raise an issue on the damaging of my goods which was ordered in april 2010 and was deliverd in 2 days but the quality of my 1 shirt and 2 pants were damaged.i want a refund.<|im_end|>
<|im_start|>assistant
'''

In [ ]:
chat_top_template = '''
<|im_start|>system
You are Max, a virtual assistant designed to provide efficient customer support for an ecommerce website called broshop. Users rely on your assistance to find and know products in their cart, and receive clear and direct answers.
the website is a platform for sellers on broshop, a wholesale marketplace that connects buyers and sellers across India. Your website allows sellers to manage their products, orders, payments, and customer service through a simple and intuitive interface. On the left side of the homepage, there is a sidebar with icons and labels for different sections, such as:
Products = contains sections to add product(either single product or bulk product).
Orders = contains history of all the orders made by the user.
Calculator = used to do calculations.
Remittance = contains finished orders.
Help desk = used to raise an issue or checked the status of an already raised issue.
FAQ = used to see FAQ or ask a question.
Profile = user detials including personal and account details.
Logout = logs out the user.
Each option has an icon and a brief instruction to guide the sellers.
contact info = bro@mail.com +1-555-5555
hq = las vegas

If you don't know the answer to a question, please don't share false information.<|im_end|>
'''
order_top_template = '''
<|im_start|>system
you are an AI assistant that helps to extract 8 features namely title,category,mrp,selling cost,material,size,delivery date and description. from the user's input and display it to them pointwise. if any of the feature is not present in the input, you truthfully say that the info is not specified.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<|im_end|>
'''
raise_top_template = '''
<|im_start|>system
you are an AI assistant that help to extract 2 features namely category and description from the user's input and display it to them pointwise. if any of the feature is not present in the input, you truthfully say that the info is not specified.<|im_end|>
'''
def call_bot(open_template,header,user_inp)
  global x
  if x==0:
    sequences = pipeline(open_template)
    out = sequences[0]['generated_text']
    continue
  pro = f'''<|im_end|>
<|im_start|>user
{user_inp}<|im_end|>
<|im_start|>assistant
'''
  out = header+'<|im_start|>user'+out.split('<|im_start|>user')[-1]+pro
  sequences = pipeline(out)
  out = sequences[0]['generated_text']
  print(out)
  x += 1

def mailer(to,sub,cont,from_='enigmademo40@gmail.com',pass_='lsxtqkajwahrvurq'):
  # Create the MIMEText object for the email content
  msg = MIMEText(cont)
  msg["Subject"] = sub
  msg["From"] = from_
  msg["To"] = to

  # Connect to the SMTP server (in this case, Gmail's SMTP server)
  with smtplib.SMTP("smtp.gmail.com", 587) as server:
      # Start the TLS connection
      print(server.starttls())

      # Login to your email account
      print(server.login(from_, pass_))

      # Send the email
      print(server.sendmail(sender_email, to_email, msg.as_string()))
  return {"message":msg.as_string()}


In [ ]:
!pip install pyngrok flask_cors


conf.get_default().auth_token = userdata.get('nkey')
print(ngrok.connect(5000))



# Create a Flask app
app = Flask(__name__)
CORS(app)

@app.route("/",methods=['GET','POST'])
def home():
  # text = request.args.get('str')
  # id = request.args.get('user_id')
  return 'hello'

  if 'reschedule' == op.lower():
    draft_email(id,op.lower(),specific)
    ca,ad = map(lambda x: x.strip(),specific.split('-'))
    time,day = ca.split()
    ttime,tday = time[:-2]+":00 "+time[-2:].upper(),day.capitalize()
    time,day = ad.split()
    ftime,fday = time[:-2]+":00 "+time[-2:].upper(),day.capitalize()
    tt = dele_table(id,ttime,tday)
    ft =  dele_table(id,ftime,fday)
    if tt!={} and ft == {}:
      return jsonify({'to':tt,'ft':ft,"message":"sceduled"})
    else:
      return jsonify({"message":"not sceduled"})

  elif 'cancel' == op.lower():
    draft_email(id,op,specific)
    time,day = specific.split()
    time,day = time[:-2]+":00 "+time[-2:].upper(),day.capitalize()
    return jsonify(dele_table(id,time,day)|{'time':time,'day':day})

  else:
    # time,day = specific.split()
    # time,day = time[:-2]+":00 "+time[-2:].upper(),day.capitalize()
    for s,x in fetch_timetable():
      for ss in s:
        if id in ss:
          return jsonify(ss[id])
      for xx in x:
        if id in xx:
          return jsonify(xx[id])
    return "None"





@app.route('/get_pdf')
def get_pdf():
    # Replace this with your actual PDF content
    pdf_content = open('/content/')
    # Convert PDF content to base64
    pdf_base64 = base64.b64encode(pdf_content).decode('utf-8')

    # Redirect the client to a URL with the base64-encoded PDF as a parameter
    return f'<a href="/download_pdf?pdf_base64={pdf_base64}">Download PDF</a>'

@app.route('/download_pdf', methods=['GET','POST'])
def download_pdf():
    # Retrieve the base64-encoded PDF from the URL parameter
    pdf_base64 = request.args.get('pdf_base64', '')

    # Decode the base64 string
    pdf_content = base64.b64decode(pdf_base64)

    # Serve the PDF file for download
    return send_file(BytesIO(pdf_content), as_attachment=True, download_name='example.pdf')

# Run the app
app.run()


NgrokTunnel: "https://193f-34-23-59-63.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 04:38:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 04:38:16] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# Import the LangChain library
from langchain import LangChain, Response

# Create a LangChain instance
lc = LangChain()

# Define the user profile
lc.user_profile(
    name="Max",
    role="virtual assistant",
    description="Designed to provide efficient customer support for an ecommerce website called broshop, similar to Amazon in UI."
)

# Define the initial data
initial_data = '''
You are Max, a virtual assistant designed to provide efficient customer support for an ecommerce website called broshop which is similar to Amazon in UI. Users rely on your assistance to find and know products in their cart, and receive clear and direct answers. The provided data represents a user named rajesh's shopping cart with a unique user ID of 12345. The cart contains three different items with their respective details.Product 1 is red jeans with a price of $10.99 per unit and a quantity of 2.Product 2 is a black mouse priced at $5.99, and its quantity is 1.Lastly, the cart includes Product 3, an Intel i3 CPU, priced at $15.99, with a quantity of 3.Total price amounts to $74.94.

If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
'''

# Add the initial data to the LangChain instance
lc.add_data(initial_data)

# Define the chatbot template
lc.template("open_chatbot_template")

# Define the conversation loop
for x in range(10):
    if x == 0:
        # Start the conversation with the chatbot template
        lc.generate_response("generated_text", template="open_chatbot_template")
        continue

    # Get user input
    user_input = input()

    # Add user input to the conversation
    lc.add_data(f'user\n{user_input}\nassistant\n')

    # Generate assistant's response
    lc.generate_response("generated_text")

    # Print the assistant's response
    print(Response.get_text())

# Close the LangChain instance
lc.close()
